In [70]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.metrics import r2_score

In [71]:
# Load the dataset
df = pd.read_csv('/Users/jenniferyau/Documents/Deakin unit/2023 T3/SIT782/Case/left_dev_res_bldg-2.csv')


## Data Clean and Pre-processing

In [72]:
# handle missing column for "year_completed" with median
df['year_completed'].fillna(df['year_completed'].median(), inplace=True)

In [73]:
# handle missing column for "construction_year" with median
df['construction_year'].fillna(df['construction_year'].median(), inplace=True)

# handle missing column for "refurbished_year" with median
df['refurbished_year'].fillna(df['refurbished_year'].median(), inplace=True)

In [74]:
# drop column "census_year", "dweliing_type", "dweliing_number", "has_showers" , "number_of_floors_above_ground "
df.drop('census_year', axis=1, inplace=True)

In [75]:
df.drop('street_address', axis=1, inplace=True)
df.drop('geopoint', axis=1, inplace=True)
df.drop('dwelling_type', axis=1, inplace=True)
df.drop('dwelling_number', axis=1, inplace=True)
df.drop('has_showers', axis=1, inplace=True)
df.drop('property_id', axis=1, inplace=True)

In [76]:
df.drop('number_of_floors_above_ground', axis=1, inplace=True)
df.drop('predominant_space_use', axis=1, inplace=True)
df.drop('accessibility_type', axis=1, inplace=True)
df.drop('accessibility_type_description', axis=1, inplace=True)
df.drop('accessibility_rating', axis=1, inplace=True)
df.drop('bicycle_spaces', axis=1, inplace=True)

In [77]:
# put column "clue_small_area", "status" into label encoding
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
df['clue_small_area'] = labelencoder.fit_transform(df['clue_small_area'])
df['status'] = labelencoder.fit_transform(df['status'])

In [78]:
df

,status,year_completed,clue_small_area,clue_block,resi_dwellings,studio_dwe,one_bdrm_dwe,two_bdrm_dwe,three_bdrm_dwe,student_apartments,...,hospital_flr,recreation_flr,publicdispaly_flr,community_flr,car_spaces,bike_spaces,longitude,latitude,construction_year,refurbished_year
0,2,2006.0,6,342,17,0,0,0,0,0,...,0,0,0,0,0,0,144.945030,-37.802822,1890.0,2002.0
1,2,2005.0,6,333,12,0,0,0,0,0,...,0,0,0,0,0,0,144.945947,-37.802049,2005.0,2002.0
2,2,2013.0,12,414,3,0,0,0,3,0,...,0,0,0,0,0,0,144.944719,-37.806791,1970.0,2002.0
3,2,2014.0,12,409,28,0,17,11,0,0,...,0,0,0,0,0,28,144.942096,-37.806072,1888.0,2014.0
4,2,2007.0,6,1012,0,0,0,0,0,0,...,0,0,0,0,0,0,144.939286,-37.800374,1972.5,2002.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169,3,2012.0,0,245,36,0,0,20,16,0,...,0,0,0,0,56,137,144.964800,-37.802300,1983.0,2002.0
1170,3,2012.0,2,620,23,0,0,12,11,0,...,0,0,0,0,35,25,144.982577,-37.809940,1972.5,2002.0
1171,3,2012.0,10,807,372,0,140,161,71,0,...,0,0,0,0,111,153,144.959900,-37.825000,1972.5,2002.0
1172,3,2012.0,2,621,14,0,0,3,11,0,...,0,0,0,0,58,39,144.978950,-37.809569,1972.5,2002.0


In [79]:
print(df.describe())

            status  year_completed  clue_small_area   clue_block  \
count  1174.000000     1174.000000      1174.000000  1174.000000   
mean      1.830494     2012.412266         4.928450   516.663543   
std       0.562594        5.141018         3.591929   476.873074   
min       0.000000     2002.000000         0.000000     1.000000   
25%       2.000000     2010.000000         2.000000   104.000000   
50%       2.000000     2012.000000         4.000000   403.000000   
75%       2.000000     2016.000000         7.000000   781.000000   
max       3.000000     2023.000000        12.000000  2547.000000   

       resi_dwellings   studio_dwe  one_bdrm_dwe  two_bdrm_dwe  \
count     1174.000000  1174.000000   1174.000000   1174.000000   
mean        76.191652     1.161840     21.487223     26.979557   
std        155.269669     9.596961     60.291034     75.038061   
min          0.000000     0.000000      0.000000      0.000000   
25%          0.000000     0.000000      0.000000      0.0

## Linear regression

Target variable : resi_dwellings

In [80]:
# group by 'year_completed' and calculate the sum of 'resi_dwellings'
yearly_sum = df.groupby('year_completed')['resi_dwellings'].sum().reset_index()

# rename the column 
yearly_sum.rename(columns={'resi_dwellings': 'total_resi_dwellings'}, inplace=True)

In [81]:
# use 'year_completed' and 'total_resi_dwellings'
X = yearly_sum[['year_completed']]
y = yearly_sum['total_resi_dwellings']

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# predict future values
future_years = pd.DataFrame({'year_completed': np.arange(2024, 2028)})
future_predictions = model.predict(future_years)
print(future_predictions)

Mean Squared Error: 5074486.20804333
[4972.24004425 5068.61827065 5164.99649705 5261.37472345]


In [82]:
# calculate R-squared 
r_squared = r2_score(y_test, y_pred)
print(f"R-squared: {r_squared}")

R-squared: 0.127596438214857


Target variable : studio_dwe

In [83]:
# group by 'year_completed' and calculate the sum of 'stuido_dwe'
yearly_sum2 = df.groupby('year_completed')['studio_dwe'].sum().reset_index()

# rename the column 
yearly_sum2.rename(columns={'studio_dwe': 'total_studio_dwe'}, inplace=True)

In [84]:
# use 'year_completed' and 'total_studio_dwe' 
X = yearly_sum2[['year_completed']]
y = yearly_sum2['total_studio_dwe']

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# predict future values
future_years = pd.DataFrame({'year_completed': np.arange(2024, 2028)})
future_predictions = model.predict(future_years)
print(future_predictions)

Mean Squared Error: 3857.9033864210883
[88.26438053 89.33379425 90.40320796 91.47262168]


In [85]:
# Calculate R-squared 
r_squared = r2_score(y_test, y_pred)
print(f"R-squared: {r_squared}")

R-squared: -18.0457315680346


Target variable : one_bdrm_dwe

In [86]:
# group by 'year_completed' and calculate the sum of 'one_bdrm_dwe'
yearly_sum3 = df.groupby('year_completed')['one_bdrm_dwe'].sum().reset_index()

# rename the column 
yearly_sum3.rename(columns={'one_bdrm_dwe': 'total_one_bdrm_dwe'}, inplace=True)

In [87]:
# use 'year_completed' and 'total_one_bdrm_dwe' 
X = yearly_sum3[['year_completed']]
y = yearly_sum3['total_one_bdrm_dwe']

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# predict future values
future_years = pd.DataFrame({'year_completed': np.arange(2024, 2028)})
future_predictions = model.predict(future_years)
print(future_predictions)

Mean Squared Error: 729906.8595935808
[2083.65597345 2165.27903761 2246.90210177 2328.52516593]


In [88]:
# Calculate R-squared 
r_squared = r2_score(y_test, y_pred)
print(f"R-squared: {r_squared}")

R-squared: 0.5159747523068796


Target variable: two_bdrm_dwe

In [89]:
# group by 'year_completed' and calculate the sum of 'two_bdrm_dwe'
yearly_sum4 = df.groupby('year_completed')['two_bdrm_dwe'].sum().reset_index()

# rename the column 
yearly_sum4.rename(columns={'two_bdrm_dwe': 'total_two_bdrm_dwe'}, inplace=True)

In [90]:
# use 'year_completed' and 'total_two_bdrm_dwe'
X = yearly_sum4[['year_completed']]
y = yearly_sum4['total_two_bdrm_dwe']

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# predict future values
future_years = pd.DataFrame({'year_completed': np.arange(2024, 2028)})
future_predictions = model.predict(future_years)
print(future_predictions)

Mean Squared Error: 1105703.6497176003
[2717.70464602 2828.46008481 2939.2155236  3049.97096239]


In [91]:
# Calculate R-squared 
r_squared = r2_score(y_test, y_pred)
print(f"R-squared: {r_squared}")

R-squared: 0.5274014893229759


Target variable: three_bdrm_dwe

In [92]:
# group by 'year_completed' and calculate the sum of 'three_bdrm_dwe'
yearly_sum5 = df.groupby('year_completed')['three_bdrm_dwe'].sum().reset_index()

# rename the column 
yearly_sum5.rename(columns={'three_bdrm_dwe': 'total_three_bdrm_dwe'}, inplace=True)

In [93]:
# use 'year_completed' and 'total_three_bdrm_dwe' 
X = yearly_sum5[['year_completed']]
y = yearly_sum5['total_three_bdrm_dwe']

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# predict future values
future_years = pd.DataFrame({'year_completed': np.arange(2024, 2028)})
future_predictions = model.predict(future_years)
print(future_predictions)

Mean Squared Error: 16535.300111291883
[447.4579646  464.18455015 480.91113569 497.63772124]


In [94]:
# Calculate R-squared 
r_squared = r2_score(y_test, y_pred)
print(f"R-squared: {r_squared}")

R-squared: 0.6028086071556116


Target variable: student_apartments

In [65]:
# group by 'year_completed' and calculate the sum of 'student_apartments'
yearly_sum6 = df.groupby('year_completed')['student_apartments'].sum().reset_index()

# rename the column 
yearly_sum6.rename(columns={'student_apartments': 'total_student_apartments'}, inplace=True)

In [66]:
# use 'year_completed' and 'total_student_apartments' 
X = yearly_sum6[['year_completed']]
y = yearly_sum6['total_student_apartments']

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# predict future values
future_years = pd.DataFrame({'year_completed': np.arange(2024, 2028)})
future_predictions = model.predict(future_years)
print(future_predictions)

Mean Squared Error: 439357.29221928585
[1721.44469027 1797.78668879 1874.12868732 1950.47068584]


In [69]:
# Calculate R-squared 
r_squared = r2_score(y_test, y_pred)
print(f"R-squared: {r_squared}")

R-squared: -10.727002064283399
